# 向量儲存 Vector datastore


使用語法查詢


```sql
-- 顯示當前資料庫
SELECT current_database();

-- 顯示資料庫中的資料表
SELECT table_name
FROM postgres.information_schema.tables
WHERE table_schema = 'public';
```


導入函式庫


In [73]:
# BedrockEmbeddings 類別是用來將文本轉換成嵌入向量的工具
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores.pgvector import PGVector, DistanceStrategy
import pandas as pd
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv

# 載入環境參數
load_dotenv()

True

設定連線資訊

In [74]:
# 集合名稱
COLLECTION_NAME = "my_collection"

# 連線參數
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.getenv("PGVECTOR_DRIVER"),
    user=os.getenv("PGVECTOR_USER"),
    password=os.getenv("PGVECTOR_PASSWORD"),
    host=os.getenv("PGVECTOR_HOST"),
    port=os.getenv("PGVECTOR_PORT"),
    database=os.getenv("PGVECTOR_DATABASE"),
)

連線並查詢

In [75]:
# 建立 SQLAlchemy 引擎
engine = create_engine(CONNECTION_STRING)

# 執行查詢來獲取當前資料庫名稱
query = "SELECT current_database();"

with engine.connect() as connection:
    result = connection.execute(text(query))
    current_database = result.fetchone()[0]

print(f"當前資料庫名稱：{current_database}")

當前資料庫名稱：mydatabase


查詢當前資料表

In [76]:
# 查詢資料表名稱
query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
"""

# 建立與資料庫的連接並執行查詢
with engine.connect() as connection:
    result = connection.execute(text(query))
    tables = result.fetchall()

# 將結果轉換為 DataFrame 並顯示
df = pd.DataFrame(tables, columns=['table_name'])
print(df)

                table_name
0  langchain_pg_collection
1   langchain_pg_embedding


## 建立向量

建立操作向量的實體

In [77]:
# 創建一個實例，用來將文本轉換成嵌入向量
embeddings = BedrockEmbeddings()

# 建立 VectorDB 儲存實例
my_vector_store = PGVector(
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding_function=embeddings,
    distance_strategy=DistanceStrategy.EUCLIDEAN,
)

/Users/samhsiao/Documents/PythonVenv/envllmChatBot/lib/python3.11/site-packages/langchain_community/vectorstores/pgvector.py:322: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


查詢集合 `langchain_pg_collection` 的內容

In [78]:
# 取得環境參數
_DATABASE=os.getenv("PGVECTOR_DATABASE")
_USER=os.getenv("PGVECTOR_USER")
_PASSWORD=os.getenv("PGVECTOR_PASSWORD")
_HOST=os.getenv("PGVECTOR_HOST")
_PORT=os.getenv("PGVECTOR_PORT")

# 連接到 PostgreSQL 資料庫
engine = create_engine(
    f'postgresql+psycopg2://{_USER}:{_PASSWORD}@{_HOST}:{_PORT}/{_DATABASE}'
)

# 查詢資料表內容並將結果儲存在 DataFrame 中
query = "SELECT * FROM langchain_pg_collection;"

# 建立與資料庫的連接並執行查詢
with engine.connect() as connection:
    df = pd.read_sql(query, connection)
# 結果
print(df)

            name cmetadata                                  uuid
0  my_collection      None  30286ce9-1967-4266-96f7-1b9ee9630add


查詢資料表 `langchain_pg_embedding` 的內容

In [81]:
# 查詢資料表內容並將結果儲存在 DataFrame 中
query = "Select * from langchain_pg_embedding;"

# 建立與資料庫的連接並執行查詢
with engine.connect() as connection:
    df = pd.read_sql(query, connection)
# 結果
print(df)

                          collection_id  \
0  30286ce9-1967-4266-96f7-1b9ee9630add   
1  30286ce9-1967-4266-96f7-1b9ee9630add   
2  30286ce9-1967-4266-96f7-1b9ee9630add   

                                           embedding   document cmetadata  \
0  [1.46875,0.099609375,0.4609375,-0.038085938,-1...  新德里是印度的首都        {}   
1  [1.296875,-0.41992188,0.18457031,0.2578125,-0....     歡迎來到印度        {}   
2  [1.125,-0.20507812,-0.47070312,-0.103027344,0....   我今天要去踢足球        {}   

                              custom_id                                  uuid  
0  6eeb987e-ae83-4ac2-acd3-e502a063fe98  7c152885-a511-42f9-92c4-41237dfbc145  
1  5af37314-c3aa-4a96-92f8-3939168cce4a  8bc7792e-a58f-46d8-ae93-dfa50e34e482  
2  f95128c4-470d-4a01-9ad7-e556c4ea158f  aa60aee3-0a80-4b5c-9021-c27c52d4c8c3  


## 查詢資料表

使用 `psycopg2`

In [ ]:
import psycopg2
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

# 連接到 PostgreSQL 資料庫
conn = psycopg2.connect(
    dbname=os.getenv("PGVECTOR_DATABASE"),
    user=os.getenv("PGVECTOR_USER"),
    password=os.getenv("PGVECTOR_PASSWORD"),
    host=os.getenv("PGVECTOR_HOST"),
    port=os.getenv("PGVECTOR_PORT")
)

# 建立游標物件
cur = conn.cursor()
# 執行 SQL 查詢
cur.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
""")
# 獲取查詢結果
tables = cur.fetchall()
# 將結果轉換為 DataFrame（可選）
df = pd.DataFrame(tables, columns=['table_name'])
# 關閉游標和連接
cur.close()
conn.close()
# 結果
print(df)

使用 `SQLAlchemy`

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

_DATABASE=os.getenv("PGVECTOR_DATABASE")
_USER=os.getenv("PGVECTOR_USER")
_PASSWORD=os.getenv("PGVECTOR_PASSWORD")
_HOST=os.getenv("PGVECTOR_HOST")
_PORT=os.getenv("PGVECTOR_PORT")

# 連接到 PostgreSQL 資料庫
engine = create_engine(
    f'postgresql+psycopg2://{_USER}:'
    f'{_PASSWORD}@{_HOST}:{_PORT}/{_DATABASE}'
)
# 執行 SQL 查詢並將結果儲存在 DataFrame 中
query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
"""
# 執行 SQL 查詢並將結果儲存在 DataFrame 中
df = pd.read_sql(query, engine)
# 結果
print(df)

## 建立向量資料

建立三筆向量數據

In [80]:
texts = ["新德里是印度的首都", "歡迎來到印度", "我今天要去踢足球"]

# Text --> Embeddings --> Vectors --> Aurora
my_vector_store.from_texts(
    texts=texts,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding=embeddings,
)

/Users/samhsiao/Documents/PythonVenv/envllmChatBot/lib/python3.11/site-packages/langchain_community/vectorstores/pgvector.py:322: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


建立更多向量

In [82]:
texts = [
    "今晚的天空晴朗",
    "貓是好奇的動物",
    "巴黎正在下雨",
    "學習 Python 可以很有趣",
    "和朋友一起喝咖啡味道更好",
    "我住在波士頓，這是一個美麗的城市",
    "我家旁邊有一個博物館",
    "音樂將人們聚集在一起",
    "博物館在少數地方週一休館",
    "很少有地方博物館每週開放 7 天，就像在我的城市一樣"
]

# Text --> Embeddings --> Vectors --> Aurora
my_vector_store.from_texts(
    texts=texts,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
    embedding=embeddings,
)

/Users/samhsiao/Documents/PythonVenv/envllmChatBot/lib/python3.11/site-packages/langchain_community/vectorstores/pgvector.py:322: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


## 使用 `PGVector` 進行 `相似度搜尋`


In [83]:
my_vector_store.similarity_search(
    query="任何城市的博物館都是全天開放嗎",
    # 從搜索結果中返回最相似的4個結果
    k=4
)

[Document(page_content='很少有地方博物館每週開放 7 天，就像在我的城市一樣'),
 Document(page_content='博物館在少數地方週一休館'),
 Document(page_content='我家旁邊有一個博物館'),
 Document(page_content='我住在波士頓，這是一個美麗的城市')]

## 建立新的集合

需要建立一個 uuid 作為識別符

In [84]:
# 生成新的 UUID
new_uuid = str(uuid.uuid4())

# 插入新集合的 SQL 語句
insert_collection_query = """
INSERT INTO langchain_pg_collection (name, cmetadata, uuid)
VALUES ('new_collection', '{"info": "some metadata"}', :uuid)
RETURNING uuid;
"""

# 執行插入集合並獲取新集合的 UUID
with engine.begin() as connection:  # 使用 begin() 確保提交
    result = connection.execute(text(insert_collection_query), {"uuid": new_uuid})
    new_collection_id = result.fetchone()[0]

print(f"新集合已創建，集合 UUID 為：{new_collection_id}")

新集合已創建，集合 UUID 為：f09b2d2f-8599-44dc-9e85-7d13cc72f277


查詢所有集合

In [85]:
# 查詢所有集合的 SQL 語句
query_collections = "SELECT * FROM langchain_pg_collection;"

# 建立與資料庫的連接並執行查詢
with engine.connect() as connection:
    collections_df = pd.read_sql(query_collections, connection)

# 顯示結果
print(collections_df)

             name                  cmetadata  \
0   my_collection                       None   
1  new_collection  {'info': 'some metadata'}   

                                   uuid  
0  30286ce9-1967-4266-96f7-1b9ee9630add  
1  f09b2d2f-8599-44dc-9e85-7d13cc72f277  


先進行刪除

In [ ]:
# 刪除所有名為 `new_collection` 的集合
delete_query = """
DELETE FROM langchain_pg_collection
WHERE name = 'new_collection';
"""

with engine.begin() as connection:
    connection.execute(text(delete_query))
    print("所有名為 'new_collection' 的集合已刪除。")

刪除指定名稱的集合

In [ ]:
# 指定名稱
_collection = 'my_collection'
# 刪除
delete_query = 'DELETE FROM langchain_pg_collection'
f'WHERE name = {_collection};'

with engine.begin() as connection:
    connection.execute(text(delete_query))
    print(f"所有名為 {_collection} 的集合已刪除。")

建立時會先檢查

In [86]:
# 先檢查是否存在名為 `new_collection` 的集合
check_query = """
SELECT COUNT(*)
FROM langchain_pg_collection
WHERE name = 'new_collection';
"""

with engine.connect() as connection:
    result = connection.execute(text(check_query))
    count = result.scalar()

if count == 0:
    # 生成新的 UUID
    new_uuid = str(uuid.uuid4())

    # 插入新集合的 SQL 語句
    insert_collection_query = """
    INSERT INTO langchain_pg_collection (name, cmetadata, uuid)
    VALUES ('new_collection', '{"info": "some metadata"}', :uuid)
    RETURNING uuid;
    """

    # 執行插入集合並獲取新集合的 UUID
    with engine.begin() as connection:  # 使用 begin() 確保提交
        result = connection.execute(text(insert_collection_query), {"uuid": new_uuid})
        new_collection_id = result.fetchone()[0]

    print(f"新集合已創建，集合 UUID 為：{new_collection_id}")
else:
    print("集合 'new_collection' 已經存在，未進行插入操作。")

集合 'new_collection' 已經存在，未進行插入操作。
